In [1]:
# Acá deberían cargar todas las librerías que vayan a usar. 
# Es preferible cargar las librerías al principio para que quede claro que requerimientos tiene la notebook. 
# Tengan cuidado de no cargar librerías que no vayan a utilizar ya que esto incurre en un gasto de memoria innecesario

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [2]:
#Podriamos cargar los datos con la API de Kaggle sin necesidad de bajarlos, o alternativamente subirlos y no correr esta celda

import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c examen-2-ap-itba


import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
datos.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
resultados.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
#Levantamos los datos (si los tenemos local y no corrimos la celda anterior, habría que subirlos)

df_train=pd.read_csv("datos.csv", sep=",")
df_test=pd.read_csv("resultados.csv", sep=",")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Después deberían hacer un poco de EDA para conocer la base
df_train.columns

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers',
       'seasonNumber', 'episodeNumber', 'ordering', 'language', 'types',
       'attributes', 'isOriginalTitle', 'adult', 'budget', 'genres_y',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'video', 'id'],
      dtype='object')

In [5]:
df_train.shape

(900000, 32)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 32 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   averageRating         900000 non-null  float64
 1   numVotes              900000 non-null  int64  
 2   titleType             899998 non-null  object 
 3   isAdult               899998 non-null  float64
 4   startYear             899998 non-null  object 
 5   endYear               899998 non-null  object 
 6   runtimeMinutes        899998 non-null  object 
 7   genres_x              899997 non-null  object 
 8   directors             899998 non-null  object 
 9   writers               899998 non-null  object 
 10  seasonNumber          403675 non-null  object 
 11  episodeNumber         403675 non-null  object 
 12  ordering              341292 non-null  float64
 13  language              341292 non-null  object 
 14  types                 341292 non-null  object 
 15  

In [7]:
df_train['rd']=pd.to_datetime(df_train['release_date']).values.astype(float)

In [8]:
#Elimino los NA de las variables continuas

contin_vars = ['numVotes', 'runtimeMinutes', 'budget','revenue', 'startYear','endYear','popularity','runtime','isAdult','isOriginalTitle','adult', 'budget','revenue', 'ordering','rd', 'video', 'seasonNumber', 'episodeNumber']
for v in contin_vars: df_train.loc[df_train[v].isnull(), v] = 0
for v in contin_vars: df_train.loc[(df_train[v].isna()), v] = 0

In [9]:
df_train=df_train.replace('\\N',0)
df_train['adult']=df_train['adult'].replace('2020',0)

df_train['endYear']=df_train['endYear'].astype('int16')
df_train['startYear']=df_train['startYear'].astype('int16')
df_train['prod']=df_train['endYear']-df_train['startYear']
df_train['prod'][df_train['prod']<0] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
df_train['runtimeMinutes']=df_train['runtimeMinutes'].replace('Game-Show',0)
df_train['runtimeMinutes']=df_train['runtimeMinutes'].astype('int16')
df_train['video2']=df_train['video'].astype('int16')

df_train['RM_cuad']=df_train['runtimeMinutes']*df_train['runtimeMinutes']

In [11]:
 def label_race (row, col):
    lab = {
        f[0]: 6,
        f[1]: 5,
        f[2]: 4,
        f[3]: 3,
        f[4]: 2,
        f[5]: 1,
    }
    return lab.get(row[col], 0)

In [12]:
f=df_train['directors'].value_counts()[1:7].index.tolist()
df_train['director_f'] = df_train.apply (lambda row: label_race(row,'directors'), axis=1)

In [13]:
f=df_train['writers'].value_counts()[1:7].index.tolist()
df_train['writers_f'] = df_train.apply (lambda row: label_race(row,'writers'), axis=1)

In [14]:
f=df_train['production_companies'].value_counts()[1:7].index.tolist()
df_train['production_companies_f'] = df_train.apply (lambda row: label_race(row,'production_companies'), axis=1)

In [15]:
f=df_train['production_countries'].value_counts()[1:7].index.tolist()
df_train['production_countries_f'] = df_train.apply (lambda row: label_race(row,'production_countries'), axis=1)

In [16]:
f=df_train['status'].value_counts()[0:6].index.tolist()
df_train['status_f'] = df_train.apply (lambda row: label_race(row,'status'), axis=1)

In [17]:
#Clasifico en categóricas y continuas
cat_vars = ['language','titleType','attributes'] 
contin_vars = ['startYear','endYear','prod', 'numVotes', 'runtimeMinutes', 'budget','revenue', 'rd','director_f', 'writers_f','production_companies_f', 'production_countries_f','status_f' ,'popularity','runtime','isAdult','isOriginalTitle','adult', 'budget','revenue', 'ordering', 'video2', 'RM_cuad']

In [18]:
print(df_train.columns.tolist())

['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers', 'seasonNumber', 'episodeNumber', 'ordering', 'language', 'types', 'attributes', 'isOriginalTitle', 'adult', 'budget', 'genres_y', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'video', 'id', 'rd', 'prod', 'video2', 'RM_cuad', 'director_f', 'writers_f', 'production_companies_f', 'production_countries_f', 'status_f']


In [19]:
df_train.groupby('titleType')['averageRating'].mean()

titleType
movie           6.106495
short           6.781752
tvEpisode       7.392150
tvMiniSeries    7.088109
tvMovie         6.611189
tvSeries        6.802765
tvShort         6.855646
tvSpecial       6.759183
video           6.568889
videoGame       7.019660
Name: averageRating, dtype: float64

In [20]:
# Genero variables dummys con las categóricas
for v in cat_vars: df_train=df_train.join(pd.get_dummies(df_train[v], drop_first=False), lsuffix = '_left', rsuffix = '_right')

In [21]:
df_train['videogame episode'].value_counts()[1:6].index.tolist()

[1]

In [22]:
for a in df_train.columns:
  print(df_train.groupby(a)['averageRating'].mean())

averageRating
1.0      1.0
1.1      1.1
1.2      1.2
1.3      1.3
1.4      1.4
        ... 
9.6      9.6
9.7      9.7
9.8      9.8
9.9      9.9
10.0    10.0
Name: averageRating, Length: 91, dtype: float64
numVotes
5          7.067659
6          6.989439
7          6.937417
8          6.923146
9          6.928765
             ...   
1881862    8.900000
1912851    8.800000
2137812    8.800000
2381831    9.000000
2425542    9.300000
Name: averageRating, Length: 16711, dtype: float64
titleType
movie           6.106495
short           6.781752
tvEpisode       7.392150
tvMiniSeries    7.088109
tvMovie         6.611189
tvSeries        6.802765
tvShort         6.855646
tvSpecial       6.759183
video           6.568889
videoGame       7.019660
Name: averageRating, dtype: float64
isAdult
0.0       6.891979
1.0       6.283859
2020.0    7.700000
Name: averageRating, dtype: float64
startYear
0       7.083673
1874    6.900000
1878    6.750000
1881    5.300000
1885    5.300000
          ...   
2017  

In [23]:
#Selección de variables para el modelo
algunas=contin_vars+['en', 'es', 'yi','movie','short','tvEpisode','tvMiniSeries','tvSeries','tvMovie','tvShort','tvSpecial','complete title','new title','alternative spelling','reissue title','short title','weekend title','video box title','uncensored intended title','trailer title', 'pre-release title', 'videogame episode', 'PC version']

In [82]:
# Elegir un modelo y correrlo
# Recuerden que lo que tienen que predecir es la columna averageRating
reg=RandomForestRegressor(min_samples_split=55, n_estimators=50)

X=np.array(df_train[algunas])

y=df_train['averageRating']

res=reg.fit(X, y)

In [83]:
res.score(X,y)

0.36886608042807245

In [26]:
# Luego podría corresponder ajustar hiperparámetros, hacer CV, etc...

In [27]:
df_test['rd']=pd.to_datetime(df_test['release_date']).values.astype(float)

In [28]:
#Elimino los NA de las variables continuas
contin_vars = ['numVotes', 'runtimeMinutes', 'budget', 'revenue', 'startYear','endYear','rd', 'popularity','runtime','isAdult','isOriginalTitle','adult', 'budget','revenue', 'ordering', 'video']
for v in contin_vars: df_test.loc[df_test[v].isnull(), v] = 0
for v in contin_vars: df_test.loc[(df_test[v].isna()), v] = 0

In [29]:
df_test=df_test.replace('\\N',0)
df_train['adult']=df_train['adult'].replace('2020',0)

df_test['endYear']=df_test['endYear'].astype('int16')
df_test['startYear']=df_test['startYear'].astype('int16')
df_test['prod']=df_test['endYear']-df_train['startYear']
df_test['prod'][df_test['prod']<0] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [30]:
df_test['runtimeMinutes']=df_test['runtimeMinutes'].replace('Reality-TV',0)
df_test['runtimeMinutes']=df_test['runtimeMinutes'].astype('int16')
df_test['video2']=df_test['video'].astype('int16')
df_test['RM_cuad']=df_test['runtimeMinutes']*df_test['runtimeMinutes']

In [31]:
f=df_test['directors'].value_counts()[1:7].index.tolist()
df_test['director_f'] = df_test.apply (lambda row: label_race(row,'directors'), axis=1)

In [32]:
f=df_test['writers'].value_counts()[1:7].index.tolist()
df_test['writers_f'] = df_test.apply (lambda row: label_race(row,'writers'), axis=1)

In [33]:
f=df_test['production_companies'].value_counts()[1:7].index.tolist()
df_test['production_companies_f'] = df_test.apply (lambda row: label_race(row,'production_companies'), axis=1)

In [34]:
f=df_test['production_countries'].value_counts()[1:7].index.tolist()
df_test['production_countries_f'] = df_test.apply (lambda row: label_race(row,'production_countries'), axis=1)

In [35]:
 def label_race (row, col):
    lab = {
        f[0]: 6,
        f[1]: 5,
        f[2]: 4,
        f[3]: 3,
        f[4]: 2,
        f[5]: 1,
        f[6]: 0,
    }
    return lab.get(row[col], 0)

In [36]:
f=df_test['status'].value_counts()[0:5].index.tolist()
f.append('1')
f.append('2')
df_test['status_f'] = df_test.apply (lambda row: label_race(row,'status'), axis=1)

In [37]:
# Genero variables dummys con las categóricas
for v in cat_vars: df_test=df_test.join(pd.get_dummies(df_test[v], drop_first=False), lsuffix = '_left', rsuffix = '_right')

In [84]:
# Y hago una predicción con las variables de testeo
X=np.array(df_test[algunas])
resultado=reg.predict(X)

In [86]:
# Finalmente generar un CSV con los resultados aplicados al CSV de resultados que no contiene la columna target

salida = pd.DataFrame(data={"averageRating": resultado}).astype(str)
salida.index = df_test.id
salida.to_csv("RF.csv", sep=',',index=True,  index_label='id')

In [40]:
resul=pd.read_csv("derived.csv", sep=",")

In [85]:
res.score(X,resul.averageRating)

0.2800092173260764